# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from textblob import TextBlob as tb
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'sample_submission.csv')

df_cities = pd.read_csv('/kaggle/input/cities/cities.csv')


In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
#  переименуем колонки
data.rename(
    columns={
        "Cuisine Style": "cuisine",
        "Price Range": "price",
        "Number of Reviews": "reviews_count",
    },
    inplace=True,
)
data.columns = [name.lower() for name in data.columns]

In [ ]:
data.info()

Признаки:
- city: Город
- cuisine: Кухня
- ranking: Ранг ресторана относительно других ресторанов в этом городе
- price: Цены в ресторане в 3 категориях
- review_count: Количество отзывов
- reviews: 2 последних отзыва и даты этих отзывов
- url_ta: страница ресторана на www.tripadvisor.com
- id_ta: ID ресторана в TripAdvisor
- rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
# Мы не будем парсить сам сайт, поэтому сразу удалим url_ta и id_ta
data.drop(columns=["url_ta", "id_ta"], inplace=True)

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

# 0. Вспомогательные функции

In [ ]:
# статистика по пропускам
def show_misses():
    total = data.restaurant_id.count()
    print(f"{'Total':>15}: 100.0%: {total}")
    for column in data.columns:
        nan_count = len(data[data[column].isnull()][column])
        if nan_count:
            print(f"{column:>15}: {round(nan_count/total*100,1):>5}%: {nan_count}")

# сохранием информацию перед заполнением NaN
def store_nan(col):
    data[f"{col}_has_nan"] = data[col].isna()

# парсинг review
def review_format(review_string):
    def format_string(line, sep):
        line = re.sub(r",\s+nan", ", 'nan'", line)
        line = re.sub(r"nan,", "'nan',", line)
        line = re.sub(r"""^['"]""", "", line)
        line = re.sub(r"""['"]$""", "", line)
        line = re.sub(r"""['"],\s+['"]""", sep, line)
        return line

    if type(review_string) != str:
        return [[np.NaN, np.NaN], [np.NaN, np.NaN]]

    sep = "<r_separator>"
    r = re.match(r"\[\[(.*)\],\s+\[(.*)\]\]", review_string)
    review_text = format_string(r.group(1), sep).split(sep)
    review_date = format_string(r.group(2), sep).split(sep)
    review_date = [pd.to_datetime(dt) for dt in review_date]
    for indx, text in enumerate(review_text):
        if text == "nan":
            review_text[indx] = np.NaN
            review_date[indx] = np.NaN
    if len(review_text) == 1:
        review_text.append(np.NaN)
        review_date.append(np.NaN)
    return [review_text, review_date]

# гистограмма с основными показателями выбросов
def show_histogram(col, limit, bins=100):
    plt.subplots(figsize=(15, 10))
    median = data[col].median()
    iqr = data[col].quantile(0.75) - data[col].quantile(0.25)
    per25 = data[col].quantile(0.25)
    per75 = data[col].quantile(0.75)
    l_out = per25 - 1.5 * iqr
    r_out = per75 + 1.5 * iqr
    print(f"25 percentile: {per25}")
    print(f"75 percentile: {per75}")
    print(f"Outlier left: {l_out}")
    print(f"Outlier left count: {len(data[data[col]<l_out])}")
    print(f"Outlier right: {r_out}")
    print(f"Outlier right count: {len(data[data[col]>r_out])}")

    data[col].loc[data[col].between(per25 - 1.5 * iqr, per75 + 1.5 * iqr)].hist(
        bins=bins, range=(0, limit), label="Values",
    )
    data[col].loc[data[col] > r_out].hist(
        alpha=0.5, bins=bins, range=(0, limit), label="Right Outliers"
    )
    data[col].loc[data[col] < l_out].hist(
        alpha=0.5, bins=bins, range=(0, limit), label="Left Outliers"
    )    
    plt.legend()

## 1. Обработка NAN
посмотрим на число пропусков и их распределение

In [ ]:
show_misses()

In [ ]:
plt.subplots(figsize=(12, 7))
sns.heatmap(data.isnull())

In [ ]:
# Почистим дубликаты, если есть
data.duplicated(keep=False).sum()

### 2. Обработка review 

In [ ]:
# в колонке reviews много пустых записей в виде пстого списка
len(data[data.reviews=="[[], []]"])

In [ ]:
# заменим их на nan
data.reviews = data.reviews.replace("[[], []]", np.NaN)

In [ ]:
# и посмотрим на пропуси теперь
show_misses()

In [ ]:
# форматируем reviews, на выходе список списков вида [[отзыв1, отзыв2], [дата1, дата2]]
data["reviews_formatted"] = data.reviews.apply(review_format)

In [ ]:
# лист с текстами и лист в датами в отдельные временные столбцы
data[['reviews_text_tmp','reviews_date_tmp']] = pd.DataFrame(data.reviews_formatted.tolist(), index= data.index)

# временные столбцы разбиваем еще раз, каждому отзыву/дате свой стобец
data[['review_1_text','review_2_text']] = pd.DataFrame(data.reviews_text_tmp.to_list(), index= data.index)
data[['review_1_date','review_2_date']] = pd.DataFrame(data.reviews_date_tmp.to_list(), index= data.index)

# удаляем временны столбцы
data.drop(columns=['reviews_text_tmp', 'reviews_date_tmp'], inplace=True)

In [ ]:
# перед заполнением nan сохраним информацию где были пропуски
store_nan("review_1_text")
store_nan("review_2_text")
store_nan("review_1_date")
store_nan("review_2_date")

In [ ]:
# для определения тональности нужен текст, меняем NaN на ""
data.review_1_text.replace(np.NaN, "", inplace=True)
data.review_2_text.replace(np.NaN, "", inplace=True)

# и делаем еще два столбца с тональностями по каждому из ответов
data["review_1_polarity"] = data.review_1_text.apply(lambda x: tb(x).sentiment.polarity).round(2)
data["review_2_polarity"] = data.review_2_text.apply(lambda x: tb(x).sentiment.polarity).round(2)

In [ ]:
# формируем временные df с тональностями, отфильтровывая те записи, где отзыва не было
df_polarity_1 = data[data.review_1_text_has_nan==False][["city", "review_1_polarity"]]
df_polarity_2 = data[data.review_2_text_has_nan==False][["city", "review_2_polarity"]]

# меняем имя столбцов
df_polarity_1.columns = ["city", "polarity"]
df_polarity_2.columns = ["city", "polarity"]

# и объединяем оба df в один, что бы вычислить медиану по каждому городу
df_polarity = pd.concat([df_polarity_1, df_polarity_2], ignore_index=True)
median_polarity = df_polarity.groupby("city")["polarity"].median().to_dict()

In [ ]:
# восстанавливаем NaN там, где отзывов изначально не было.
data.loc[data.review_1_text_has_nan==True, "review_1_text"] = np.NaN
data.loc[data.review_2_text_has_nan==True, "review_2_text"] = np.NaN

# полярность на "" равна 0, поэтому их тоже меняем на NaN и будем заполнять вышенайденной медианов
data.loc[data.review_1_text_has_nan==True, "review_1_polarity"] = np.NaN
data.loc[data.review_2_text_has_nan==True, "review_2_polarity"] = np.NaN

In [ ]:
# заполняем NaN медианным значением по каждому городу
data.review_1_polarity.fillna(data.city.map(median_polarity), inplace=True)
data.review_2_polarity.fillna(data.city.map(median_polarity), inplace=True)

## 3. Обработка city 

In [ ]:
# список городов
data.city.value_counts()

In [ ]:
# посмотрим на распределение ресторанов по городам
plt.subplots(figsize=(15, 10))
plt.xticks(ticks=range(0, 7500, 500))
plt.title("Рестораны по городам")
sns.countplot(y="city", data=data, order=data.city.value_counts().index, color="royalblue")
plt.xlabel("Число ресторанов")
plt.ylabel("Город")

In [ ]:
# добавим столбцы для городов
# - population - численность населения
# - density - плотность населения
# - capital - является ли столицей
# - purchasing_power_index - индекс покупательской способности

data = data.reset_index().merge(df_cities, how="left").set_index("index")


In [ ]:
data.head()

## 4. Обработка price 

In [ ]:
# возможные значения
data.price.unique().tolist()

In [ ]:
# заменим значения на 1,2,3,nan
def price_to_int(price):
    if price == '$':
        return 1
    elif price == '$$ - $$$':
        return 2
    elif price == '$$$$':
        return 3
    else:
        return np.nan

data.price = data.price.apply(price_to_int)

In [ ]:
# посмотрим медиану по городам
data.groupby("city")["price"].median().unique().tolist()

In [ ]:
# везде медиана одинаковая, поэтому сохраняем строки, где были пропуски и заполняем медианой
store_nan("price")
data.price.fillna(data.price.median(), inplace=True)

## 5. Обработка reviews_count 

In [ ]:
# большой разброс значений
data.reviews_count.describe()

In [ ]:
# посмотрим на гистограмме
show_histogram('reviews_count', 1200)

можно попробовать логарифмировать или привязать к численности города

In [ ]:
# логорифмируем
data["reviews_count_log"] = np.log(data.reviews_count + 1)
show_histogram("reviews_count_log", 10, bins=50)

выглядит лучше, теперь сделаем привязку к численности города

In [ ]:
data["reviews_count_pop"] = data.reviews_count / data.population
show_histogram("reviews_count_pop", 1)

судя по количеству выбросов, стало хуже. Оставляем логорифмированные значения. Посмотрим boxplot еще:

In [ ]:
# удаляем reviews_count_pop
data.drop(columns=["reviews_count_pop"], inplace=True)

In [ ]:
sns.boxplot(x="reviews_count_log", data=data)

заменим выбросы и пропуски медианными значениями

In [ ]:
per75 = data.reviews_count_log.quantile(0.75)
per25 = data.reviews_count_log.quantile(0.25)
iqr = per75 - per25
r_out = per75 + 1.5 * iqr

# сохраним информацию о том, где были выбросы
data['reviews_count_log_outliers'] = data.reviews_count_log >= r_out
# сохраним информацию о том, где были пропуски
store_nan("reviews_count_log")

# сбрасываем выбросы в NaN, что бы потом заполнить медианой
data.loc[data.reviews_count_log>=r_out, "reviews_count_log"] = np.NaN

# вычисляем медиану числа отзывов для каждого города
median_reviews_count_log = data[data.reviews_count_log.notnull()].groupby("city")["reviews_count_log"].median().to_dict()

# заполняем пропуски медианой
data.reviews_count_log.fillna(data.city.map(median_reviews_count_log), inplace=True)

## 6. Обработка cuisine

In [ ]:
# заполним пропуски "unknown"
data.cuisine.fillna("['unknown']", inplace=True)

# конвертируем в list
data.cuisine = data.cuisine.apply(lambda x: eval(x))

# добавим столбец числа кухонь
data['cuisine_count'] = data.cuisine.apply(lambda x: len(x))

In [ ]:
# помотрим еще на наличие пропусков
show_misses()

## 7. Обработка ranking

In [ ]:
# Посмотрим распредение ресторанов по рангу
show_histogram("ranking", 20000)

In [ ]:
# посмотрим на данные по ranking топ 10-ти городов
for x in (data.city.value_counts())[0:10].index:
    data.ranking[data.city == x].hist(bins=100, figsize=(15,5))

В больших городах больше ресторанов, значит Ranking у некоторых может быть очень высокий. Нужно нормировать значения по городам.

In [ ]:
data["ranking_norm"] = data.groupby("city").ranking.apply(lambda x: x/x.max())

In [ ]:
# снова смотрим гистограмму
for x in (data.city.value_counts())[0:10].index:
    data.ranking_norm[data.city == x].hist(bins=100, figsize=(15,5))


# Feature Engineering

In [ ]:
# dummy-признаки городов
dummy_city = pd.get_dummies(data.city)
data = pd.concat([data, dummy_city], axis=1)

In [ ]:
# отношение относительного ранга к населению
data['ranking_to_population'] = data.ranking / data.population * 1000

# отношение населения к число отзывов
data['population_to_reviews_count'] = data.population / data.reviews_count_log

In [ ]:
# удаляем обработанные или ненужные стобцы
data = data.drop(
    [
        "city",
        "restaurant_id",
        "reviews",
        "reviews_count",
        "reviews_formatted",
        "cuisine",
        "review_1_text",
        "review_2_text",
        "review_1_text_has_nan",
        "review_2_text_has_nan",
        "review_1_date",
        "review_2_date",
        "review_1_date_has_nan", 
        "review_2_date_has_nan",
        "price_has_nan",
    ], 
    axis = 1,
)


In [ ]:
# тепловая карта корреляции признаков

sns.set(font_scale=0.75)
plt.subplots(figsize=(25, 20))
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True, fmt='.2f', linewidths=0.1, cmap="coolwarm")

Наибольшая корреляця с целевой переменной у ranking_log (оно и понятно), но все равно связь очень низкая. Поэтому оставляем все признаки.

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# Обучение модели¶

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# выделим датасет для обучения
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values              # наш таргет
X = train_data.drop(['rating'], axis=1)

# разделим на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED)
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# округляем до 0.5 полученные значения
y_pred = (y_pred * 2).round()/2
y_pred

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data

In [ ]:
test_data = test_data.drop(['rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission = (predict_submission * 2).round()/2
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
